In [ ]:
 

from flask import Flask, request, jsonify
import cv2
import numpy as np
from tensorflow.keras.models import load_model



app = Flask(__name__)
model = load_model("6food.h5")
classes = ('fresh', 'rotten')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the input image from the request
        file = request.files['image']
        
        if not file:
            return jsonify({'error': 'No image provided'})

        # Convert the image to a NumPy array
        image_stream = file.read()
        image_array = np.frombuffer(image_stream, dtype=np.uint8)

        if len(image_array) == 0:
            return jsonify({'error': 'Invalid image format'})

        # Decode the image
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (128, 128)) / 255.0
        image = np.reshape(image, (1, 128, 128, 3))

        # Make a prediction using the trained model
        prediction = model.predict(image.reshape((1, 128, 128, 3)))
        predicted_class = np.argmax(prediction, axis=1)[0]
        predicted_label = classes[predicted_class]

        # Prepare the response
        response = {"prediction": predicted_label}

        return jsonify(response)

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0')  


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.3.76:5000
Press CTRL+C to quit
